# Init

In [287]:
from decouple import Config, RepositoryEnv
from markdownify import markdownify as md
import pandas as pd
import plotly.express as px
import requests
import json

In [288]:
config = Config(RepositoryEnv('.env'))

ollama_base_url = config('OLLAMA_API_URL')

ollama_api_endpoint = f"{ollama_base_url}/api/generate/"

ollama_bearer_token = config('OLLAMA_BEARER_TOKEN')

In [289]:
import requests
import json

def simple_ollama_request(
        prompt_system: str,
        prompt_user: str,
        ollama_api_endpoint: str,
        model_options: dict = None,  # Default to None
        request_options: dict = None,  # Default to None
        response_format: str = None,
        seed: int = None,
        model: str = "llama3.2:1b",
        bearer_token: str = None
    ):
    
    # Ensure options is a dictionary (default empty dictionary if None)
    if model_options is None:
        model_options = {}

    if request_options is None:
        request_options = {}
    
    headers = {'Content-Type': 'application/json'}
    
    # Handle the bearer token with special character encoding
    if bearer_token:
        utf8_bytes = f'Bearer {bearer_token}'.encode('utf-8')
        auth_header_value = utf8_bytes.decode('latin1')
        headers['Authorization'] = auth_header_value

    if response_format not in ['json',None]:
        print('Please select a correct output format, json or None. App select None by default')
        response_format = None
    # Build the data payload
    data = {
        "model": model,
        "system": prompt_system,
        "prompt": prompt_user,
        "stream": False,
        "format": response_format,
        "options": {
            "seed": seed,
            "top_k": model_options.get('top_k', 20),
            "top_p": model_options.get('top_p', 0.9),
            "temperature": model_options.get('temperature', 0.8),
            "repeat_penalty": model_options.get('repeat_penalty', 1.2),
            "presence_penalty": model_options.get('presence_penalty', 1.5),
            "frequency_penalty": model_options.get('frequency_penalty', 1.0),
            "num_ctx": request_options.get('num_ctx', 16384),
            "num_predict": request_options.get('num_predict', 32),
            "batch_size":128,
        }
    }

    try:
        response = requests.post(ollama_api_endpoint, data=json.dumps(data), headers=headers)
        # print(f"Status Code: {response.status_code}")
        # print(f"Response Text: {response.text}")
        return response.json() if response.status_code == 200 else None
    except Exception as error:
        print('-----------------------------------------')
        print('Something went wrong when calling Ollama')
        print(error)
        return None

In [290]:
# context option are based on https://github.com/NVIDIA/RULER
# If model is not on doc we take the nearest one

# best context is based on  Effective length
# max context is base on claimed length

mistral_nemo_model_options = {
    "name":"mistral-nemo:latest",
    "best_context":16000,
    "max_context":128000,
    "top_k": 10,
    "top_p": 0.9,
    "temperature": 0.6,
    "repeat_penalty": 1.2,
    "presence_penalty": 1.5,
    "frequency_penalty": 1.0,
}

mistral_small_options = {
    "name":"mistral-small:latest",
    "best_context":32000,
    "max_context":64000,
    "top_k": 10,
    "top_p": 0.9,
    "temperature": 0.6,
    "repeat_penalty": 1.2,
    "presence_penalty": 1.5,
}

qwen2_5_14b_model_options = {
    "name":"qwen2.5:14b",
    "best_context":32000,
    "max_context":128000,
    "top_k": 20,
    "top_p": 0.9,
    "temperature": 0.8,
    "repeat_penalty": 1.2,
    "presence_penalty": 1.5,
    "frequency_penalty": 1.0,
}

qwen2_5_32b_model_options = {
    "name":"qwen2.5:32b",
    "best_context":32000,
    "max_context":128000,
    "top_k": 20,
    "top_p": 0.9,
    "temperature": 0.8,
    "repeat_penalty": 1.2,
    "presence_penalty": 1.5,
    "frequency_penalty": 1.0,
}

llama3_2_1b_model_options = {
    "name":"llama3.2:1b",
    "best_context":64000,
    "max_context":128000,
    "top_k": 20,
    "top_p": 0.9,
    "temperature": 0.8,
    "repeat_penalty": 1.2,
    "presence_penalty": 1.5,
    "frequency_penalty": 1.0,
}

llama3_1_8b_model_options = {
    "name":"llama3.1:latest",
    "best_context":64000,
    "max_context":128000,
    "top_k": 20,
    "top_p": 0.9,
    "temperature": 0.8,
    "repeat_penalty": 1.2,
    "presence_penalty": 1.5,
    "frequency_penalty": 1.0,
}

In [292]:
model = llama3_1_8b_model_options['name']
model_options = llama3_1_8b_model_options

# model = qwen2_5_14b_model_options['name']
# model_options = qwen2_5_14b_model_options

# model = mistral_nemo_model_options['name']
# model_options = mistral_nemo_model_options

# Scoring generation

In [294]:
prompt_system = """Tu aides l'utilisateur à déterminer la compatibilité d'une aide ou subvention par rapport à la description de son projet.

Instructions :

Attribue une note sous forme d'entier entre -5 et 5, selon les critères suivants :

5 : L'aide ou la subvention correspond parfaitement aux objectifs et aux besoins du projet.
 Les objectifs de l'aide/subvention sont totalement alignés avec les besoins du projet.
 Les conditions et exigences de l'aide/subvention sont complètement satisfaites par le projet.

1 : L'aide ou la subvention correspond très bien aux objectifs et aux besoins du projet.
 Les objectifs de l'aide/subvention sont majoritairement alignés avec les besoins du projet.
 La plupart des conditions et exigences de l'aide/subvention sont satisfaites par le projet.

0 : L'aide ou la subvention correspond partiellement aux objectifs et aux besoins du projet.
 Les objectifs de l'aide/subvention sont partiellement alignés avec les besoins du projet.
 Certaines conditions et exigences de l'aide/subvention sont satisfaites par le projet.

-1 : L'aide ou la subvention a une correspondance minimale avec les objectifs et les besoins du projet.
 Les objectifs de l'aide/subvention sont rarement alignés avec les besoins du projet.
 Peu de conditions et exigences de l'aide/subvention sont satisfaites par le projet.

-5 : La description du projet est insuffisamment détaillée ou trop vague, ou l'aide ou la subvention n'a aucune correspondance avec les objectifs et les besoins du projet.
 Les objectifs de l'aide/subvention ne sont pas du tout alignés avec les besoins du projet.
 Aucune des conditions et exigences de l'aide/subvention n'est satisfaite par le projet.

Réponds uniquement avec une note entre -5 et 5 sous forme d'entier. Ne fournis aucune explication, aucun mot supplémentaire, ni aucun caractère additionnel dans ta réponse.
"""

In [295]:
data = pd.read_csv("hard-database/data_at_select.csv",index_col='id')

In [296]:
with open("project-description-sample.json",'r') as file:
    project_descrpition_list = json.load(file)

In [297]:
def gen_prompt_user_sub_scoring(sub_description,sub_eligibility,project_description):
    prompt_user = f"""
    Analyse les informations suivantes pour déterminer la compatibilité entre l'aide ou la subvention et le projet de l'utilisateur.
    
    - **Aide ou subvention à analyser :**
    {sub_description}
    
    - **Critères potentiels d'éligibilité :**
    {sub_eligibility}
    
    - **Projet de l'utilisateur :**
    {project_description}
    
    **Instructions :**

    - Utilise ces informations pour attribuer une note selon les critères précédemment définis. Ne me renvoie que la note sans aucunes explications"""

    return prompt_user

In [299]:
seed_number = 4
max_retry = 3
row_list = []


score_sub_request_options = {
    "num_ctx": 16384,
    "num_predict": 8
}

request_options = score_sub_request_options

for project_descrpition_key in project_descrpition_list:
    project_description = project_descrpition_list[project_descrpition_key]
    print('-------------------------')
    print(project_description)
    for i, row in data[:30].iterrows():
        print('---------')
        sub_description = row['description']
        sub_eligibility = row['eligibility']
        prompt_user = gen_prompt_user_sub_scoring(sub_description,sub_eligibility,project_description)

        score_sub = 0
        seed = 0
        scoring_made = 0
        retry = 0
        while scoring_made < seed_number and retry < max_retry:
            seed += 1
            response = simple_ollama_request(
                prompt_system=prompt_system,
                response_format=None,
                prompt_user=prompt_user,
                ollama_api_endpoint=ollama_api_endpoint,
                bearer_token=ollama_bearer_token,
                model_options = model_options,  # Default to None
                request_options= request_options,  # Default to None
                seed=seed,
                )
            if response:
                try :
                    score_seed = int(response['response'])
                    if score_seed > 5:
                        score_seed =5
                    if score_seed < -5:
                        score_seed = -5
                    score_sub+=score_seed
                    retry = 0
                    scoring_made+=1
                    print(score_sub)
                except Exception as error:
                    retry += 1
                    # print(error)
        row['project'] = project_description
        row['project_score'] = score_sub
        row_list.append(row)

-------------------------
Revitalisation d'une zone humide
---------
---------
-5
-5
-10
-6
---------
-1
-1
-2
3
---------
-1
-1
-2
0
---------
---------
-1
-1
-2
2
---------
---------
-5
-5
-10
-10
---------
---------
-1
3
2
-3
---------
-1
-1
-2
-2
---------
-5
-5
-6
-1
---------
-5
-10
-8
-9
---------
---------
-1
-2
-2
-7
---------
-1
-1
-6
-6
---------
-5
-10
-10
-11
---------
-5
0
-5
-3
---------
0
-5
-6
-11
---------
-5
0
-5
-1
---------
5
0
-5
-5
---------
-5
-5
-10
-7
---------
5
4
8
7
---------
-1
4
3
3
---------
5
0
4
-1
---------
-1
4
3
8
---------
-1
3
2
7
---------
---------
---------
-1
-1
-2
2
-------------------------
Entretient d'un vieux moulin
---------
---------
-5
-5
-10
-6
---------
-5
0
-5
0
---------
-1
-1
-2
-7
---------
---------
-1
-1
-2
2
---------
-3
2
2
-3
---------
---------
2
7
---------
-1
-2
2
-3
---------
-5
-10
-10
-15
---------
-5
0
-5
0
---------
-5
-10
-11
-12
---------
---------
-1
-1
-2
-2
---------
-5
-5
-10
-10
---------
5
0
-5
-5
---------
-

# Question generation

In [301]:
data_score_sample = pd.DataFrame(row_list)

In [308]:
data_score_sample.to_csv("hard-database/data_project_scoring.csv",index=False)

In [310]:
data_score_sample_select = data_score_sample[data_score_sample['project_score']>5]

In [361]:
num_question = 3

prompt_system = f"""Tu aides l'utilisateur à déterminer la compatibilité de l'aide ou de la subvention à analyser par rapport à la description de son projet.

L'utilisateur a besoin que tu lui poses {num_question} questions en lien avec l'aide ou la subvention et son projet.

Ces questions doivent être ouvertes, courtes et simple et doivent l'aider à mieux décrire son projet en fonction de l'aide.

N'hésite pas à utiliser des exemples pour illustrer.

Evite les questions sur les financements et concentre toi plus sur le projet, et cherche à obtenir plus de détails.

Tu répondras au format JSON suivant : {{"Q1": str, "Q2": str, "Q3": str}}"""

In [319]:
row = data_score_sample_select[['description','eligibility','project']].iloc[0]

sub_description,sub_eligibility,project_description = row['description'],row['eligibility'],row['project']

In [344]:
def gen_prompt_user_question(sub_description,sub_eligibility,project_description,num_question=3):
    prompt_user = f"""
    Analyse les informations suivantes pour déterminer la compatibilité entre l'aide ou la subvention et le projet de l'utilisateur.
    
    - **Aide ou subvention à analyser :**
    {sub_description}
    
    - **Critères potentiels d'éligibilité :**
    {sub_eligibility}
    
    - **Projet de l'utilisateur :**
    {project_description}
    
    **Instructions :**
    
    Donne-moi {num_question} questions en lien avec la subvention et la description du projet.
    
    Tu répondras au format JSON suivant : {{"Q1": str, "Q2": str, "Q3": str}}"""

    return prompt_user

In [374]:
num_question = 3
seed = 0

score_sub_request_options = {
    "num_ctx": 16384,
    "num_predict": 1024
}

request_options = score_sub_request_options

prompt_user = gen_prompt_user_question(sub_description,sub_eligibility,project_description,num_question)

response = simple_ollama_request(
    prompt_system=prompt_system,
    response_format="json",
    prompt_user=prompt_user,
    ollama_api_endpoint=ollama_api_endpoint,
    bearer_token=ollama_bearer_token,
    model_options = model_options,  # Default to None
    request_options= request_options,  # Default to None
    seed=seed,
    )

In [357]:
len(response['context'])

1924

In [372]:
json_text = response['response']

# Remplacer les caractères HTML par une apostrophe
json_text = json_text.replace('&#39;', "'")

# Convertir la chaîne en dictionnaire Python
response_json = json.loads(json_text)


In [375]:
response_json

{'Q1': "Quel est le principal objectif de votre projets revitalisation d'une zone humide ?\n",
 'Q2': "Dans quelle région et dans quel secteur ou territoire se situez-vous pour impliquer l'aide à cette opération.\n",
 'Q3': "Pourriez vous décrire brièvement le projet de revitalization, son objectif principal?\nPuisque je ne suis pas en train d'aborder les critères du programme LEADER et que j'ai besoin plus des informations pour savoir si ma demande est viable. Vous pouvez donc me donner l’ensemble pertinent sur votre type opération ?\n"}